# Pytorch Text - Translation With A Sqeuence To Sequence Network and Attention
Notebook for following along with Pytorch Text NLP From Scratch tutorials that is looking at creating a NN to translate French to English [Pytorch](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)  website tutorial. <br><br>
The data can be downloaded [here](https://download.pytorch.org/tutorial/data.zip).

### Choices for data

<br>

### Libaries and Modules
Importing the necessary libaries and modules for the notebook.

In [1]:
#Import cell
import glob
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import math
import numpy as np
import os
import pandas as pd
import pickle as pk
import random
import re
import string
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import unicodedata

from __future__ import unicode_literals, print_function, division
from io import open
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}. Cuda available: {torch.cuda.is_available()}")
print("Imports complete")

Device: cpu. Cuda available: False
Imports complete


<br>

### Importing and preparing data sets
Importing and preparing the data for the models.

In [2]:
#Gather datasets and prepare them for consumption
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c)!= 'Mn')

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

print("Data sets functions defined.")

Data sets functions defined.


In [3]:
#Importing data sets
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
                read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs    
    
print("\nData sets successfully imported.")


Data sets successfully imported.


In [4]:
#Loader definitions

print(f"Loaders defined, running on device: {device}")

Loaders defined, running on device: cpu


In [5]:
#Setting seed value
torch.manual_seed(1247)

<br>

### Class Definitions
<b>Classes:</b><br>
<ul>
    <li>Lang - Language encoder that converts words into a single hot-vector.</li>
    <li>Encoder - Outputs some value for every word from the input sentence. For every input word the encord outputs a vector and a hidden state, which is used for the next input word.</li>
    <li>Decoder - takes the encoder output vector(s) and outputs a sequence of words to create the translation.</li>
</ul>

In [6]:
#Constants
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [7]:
#Class definition cell
class Lang:
    def __init__(self, name) -> None:
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2 #Count SOS and EOS
        return None
        
    def addSentence(self, sentence) -> None:
        for word in sentence.split(' '):
            self.addWord(word)
        return None
    
    def addWord(self, word) -> None:
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        return None

    
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size) -> None:
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        return None
    
    def forward(self, input, hidden):
        embedding = self.embedding(input).view(1, 1, -1)
        output = embedding
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size) -> None:
        super(DecoderRNN, self).init()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        return None
    
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        return None
    
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        ebeded = self.dropout(embedded)
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

print("Classes defined.")

Classes defined.


<br>

### Calculation functions
<b>Functions:</b><br>
<ul>
    <li>filterPair - Checks if the data is below the length of MAX_LENGTH</li>
    <li>filterPairs - returns elements in a list if below the MAX_LENGTH</li>
    <li>prepareData - filters data by MAX_LENGTH, printing relevant information</li>
</ul>

In [8]:
#Calculation functions cell
def filterPair(p) -> bool:
    return len(p[0].split(' ')) < MAX_LENGTH and \
            len(p[1].split(' ')) < MAX_LENGTH and \
            p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs\n" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs\n" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words, "\n")
    return input_lang, output_lang, pairs
    
print("Calculation functions defined.")

Calculation functions defined.


<br>

### Plotting functions
<b>Functions:</b>
<ul>
    <li>showPlot - plots input array, used for plot_losses</li>
</ul>

In [9]:
#Plotting functions Cell
#plt.switch_backend('agg')
%matplotlib inline

def showPlot(points) -> None:
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.title("Average Training Loss")
    plt.ylabel("Average Loss Value")
    plt.xlabel("Loss Check Point")
    return None
    
print("Plotting functions defined.")

Plotting functions defined.


<br>

### Main code

In [10]:
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ")

In [11]:
input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs

Trimmed to 10599 sentence pairs

Counting words...
Counted words:
fra 4345
eng 2803 

['il est tres fier de sa moto trafiquee .', 'he s very proud of his custom motorcycle .']


#### Training Functions

In [12]:
#Training Preparation Functions
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1,1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang,pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def asMinutes(s):
    return '%dm %ds' % (s//60, s%60)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s/percent
    rs = es - s
    return '%s (est: %s)' % (asMinutes(s), asMinutes(rs))

#### Evaluating Functions

In [13]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                    encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
            
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()
            
        return decoded_words, decoder_attentions[:di+1]

    
def evaluateRandomly(encoder, decoder, n=10) -> None:
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
    return None

### Reloading the Model

In [14]:
with open(f'{input_lang.name}_{output_lang.name}_meta.txt', 'r') as f:
    fileContents = f.read()
    f.close()
   
fileContents = [lineRead.split(' ') for lineRead in fileContents.split('\n')]
fileContents.pop() #removes empty end line

for descript, value in fileContents:
    match descript:
        case 'hidden_size:':
            hidden_size = int(value)
            print(f"Hidden_size: {hidden_size}")
        case 'input_lang.n_words:':
            assert input_lang.n_words == int(value)
            print(f"Input language name: {input_lang.name}")
        case 'output_lang.n_words:':
            print(f"Output language name: {output_lang.name}")
            assert output_lang.n_words == int(value)
        case 'Attn_additional: dropout_p=':
            dropout_pVal = int(value)
            print(f"droppit_pVal: {dropout_pVal}")
        case _:
            continue

Hidden_size: 256
Input language name: fra
Output language name: eng


In [15]:
encoderReload = EncoderRNN(input_lang.n_words, hidden_size)
encoderReload.load_state_dict(torch.load(f'encoder_{input_lang.name}_{output_lang.name}_weights.pth'))
encoderReload.eval()

EncoderRNN(
  (embedding): Embedding(4345, 256)
  (gru): GRU(256, 256)
)

In [16]:
attn_decoderReload = AttnDecoderRNN(hidden_size, output_lang.n_words,
                               dropout_p=0.1)
attn_decoderReload.load_state_dict(torch.load(f'attn_decoder_{input_lang.name}_{output_lang.name}_weights.pth'))
attn_decoderReload.eval()

AttnDecoderRNN(
  (embedding): Embedding(2803, 256)
  (attn): Linear(in_features=512, out_features=10, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=2803, bias=True)
)

### Reload Test

In [17]:
evaluateRandomly(encoderReload, attn_decoderReload)

> tu es intrepide .
= you re fearless .
< you re fearless . <EOS>

> je suis tellement malade .
= i am so sick .
< i m so sick . <EOS>

> il est tres honnete .
= he is very honest .
< he is very honest . <EOS>

> il est tres courageux .
= he is very brave .
< he is very brave . <EOS>

> j y suis habitue .
= i m accustomed to this .
< i m used to it . <EOS>

> on va bien rigoler .
= we re gonna have a lot of fun .
< we re getting out . <EOS>

> on n est jamais trop vieux pour apprendre .
= you re never too old to learn .
< you re never too old to learn . <EOS>

> je suis retenue en otage .
= i m being held hostage .
< i m being held hostage . <EOS>

> je suis en train de perdre mon temps .
= i am wasting my time .
< i m wasting my time . <EOS>

> je ne sais pas bien nager .
= i am poor at swimming .
< i m in a afraid . <EOS>



In [18]:
evaluateRandomly(encoderReload, attn_decoderReload)

> il agit en son nom propre .
= he is acting on his own behalf .
< he is acting on his own . <EOS>

> il exerce sur elle du chantage .
= she is being blackmailed by him .
< he is at it all him . <EOS>

> je ne suis pas sure d aimer ca .
= i m not sure i like this .
< i m not sure i like this . <EOS>

> vous en etes en partie responsables .
= you are in part responsible for it .
< you are in part responsible for it . <EOS>

> vous etes tres drole .
= you re very funny .
< you re very funny . <EOS>

> je suis fatiguee de boston .
= i m tired of boston .
< i m tired of boston . <EOS>

> je suis affame et assoiffe .
= i m hungry and thirsty .
< i m hungry and thirsty . <EOS>

> je suis pret a mourir .
= i am ready to die .
< i am ready to die . <EOS>

> je sur reagis .
= i m overreacting .
< i m kidding . <EOS>

> elle est toujours soigneusement habillee .
= she is always neatly dressed .
< she is always neatly dressed . <EOS>



In [19]:
print(input_lang.index2word)

{0: 'SOS', 1: 'EOS', 2: 'j', 3: 'ai', 4: 'ans', 5: '.', 6: 'je', 7: 'vais', 8: 'bien', 9: 'ca', 10: 'va', 11: 'suis', 12: 'gras', 13: 'gros', 14: 'en', 15: 'forme', 16: 'touche', 17: '!', 18: 'touchee', 19: 'malade', 20: 'triste', 21: 'timide', 22: 'mouille', 23: 'mouillee', 24: 'il', 25: 'est', 26: 'revenu', 27: 'me', 28: 'revoila', 29: 'chauve', 30: 'occupe', 31: 'occupee', 32: 'calme', 33: 'froid', 34: 'fini', 35: 'tout', 36: 'libre', 37: 'disponible', 38: 'repu', 39: 'rassasie', 40: 'content', 41: 'chez', 42: 'moi', 43: 'retard', 44: 'paresseux', 45: 'faineant', 46: 'paresseuse', 47: 'faineante', 48: 'porte', 49: 'securite', 50: 'certain', 51: 'sur', 52: 'sure', 53: 'grande', 54: 'mince', 55: 'ordonne', 56: 'ordonnee', 57: 'laid', 58: 'laide', 59: 'faible', 60: 'vieux', 61: 'dj', 62: 'bon', 63: 'riche', 64: 'ici', 65: 'flic', 66: 'un', 67: 'homme', 68: 'seule', 69: 'seul', 70: 'arme', 71: 'armee', 72: 'reveille', 73: 'aveugle', 74: 'fauche', 75: 'fou', 76: 'folle', 77: 'gueri', 78:

In [20]:
print(output_lang.index2word)

{0: 'SOS', 1: 'EOS', 2: 'i', 3: 'm', 4: '.', 5: 'ok', 6: 'fat', 7: 'fit', 8: 'hit', 9: '!', 10: 'ill', 11: 'sad', 12: 'shy', 13: 'wet', 14: 'he', 15: 's', 16: 'am', 17: 'back', 18: 'bald', 19: 'busy', 20: 'calm', 21: 'cold', 22: 'done', 23: 'fine', 24: 'free', 25: 'full', 26: 'glad', 27: 'home', 28: 'late', 29: 'lazy', 30: 'okay', 31: 'safe', 32: 'sick', 33: 'sure', 34: 'tall', 35: 'thin', 36: 'tidy', 37: 'ugly', 38: 'weak', 39: 'well', 40: 'is', 41: 'old', 42: 'a', 43: 'dj', 44: 'good', 45: 'rich', 46: 'here', 47: 'cop', 48: 'man', 49: 'alone', 50: 'armed', 51: 'awake', 52: 'blind', 53: 'broke', 54: 'crazy', 55: 'cured', 56: 'drunk', 57: 'dying', 58: 'early', 59: 'first', 60: 'fussy', 61: 'going', 62: 'loyal', 63: 'lucky', 64: 'lying', 65: 'quiet', 66: 'ready', 67: 'right', 68: 'sober', 69: 'sorry', 70: 'stuck', 71: 'timid', 72: 'tired', 73: 'tough', 74: 'yours', 75: 'she', 76: 'hot', 77: 'we', 78: 're', 79: 'kind', 80: 'poor', 81: 'swiss', 82: 'smart', 83: 'human', 84: 'french', 85: 

### Exporting Language Models

In [24]:
with open(f'{input_lang.name}_index2word.txt', 'w') as f:
    f.write(str(input_lang.index2word))
    f.close()
    
with open(f'{output_lang.name}_index2word.txt', 'w') as f:
    f.write(str(output_lang.index2word))
    f.close()

<br>